In [ ]:
!pip install streamlit

# **Section 1: Real-Time Bid/Ask Data Fetcher (OANDA API)**

In [27]:
import requests
import pandas as pd
from typing import Union, List

# OANDA API credentials
ACCESS_TOKEN = "INSERT_OANDA_ACCESS_TOKEN"
ACCOUNT_ID   = "INSERT_OANDA_ACCOUNT_ID"
OANDA_API_URL = "https://api-fxpractice.oanda.com/v3"
HEADERS = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

**Function: get_oanda_bid_ask()**

In [5]:
def get_oanda_bid_ask(pairs: Union[str, List[str]]) -> pd.DataFrame:
    pairs = [pairs] if isinstance(pairs, str) else pairs
    valid_symbols = {"AUD_USD", "EUR_USD", "USD_JPY", "GBP_USD", "EUR_JPY"}
    pairs = [p for p in pairs if p in valid_symbols]
    if not pairs:
        return pd.DataFrame()

    try:
        response = requests.get(
            f"{OANDA_API_URL}/accounts/{ACCOUNT_ID}/pricing",
            headers=HEADERS,
            params={"instruments": ",".join(pairs)}
        )
        response.raise_for_status()
        data = response.json()

        return pd.DataFrame([
            {
                "Pair": quote["instrument"].replace("_", "/"),
                "Bid": round(float(quote["bids"][0]["price"]), 6),
                "Ask": round(float(quote["asks"][0]["price"]), 6)
            }
            for quote in data.get("prices", [])
        ])

    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return pd.DataFrame()


In [6]:
def compute_cross_rate(df, base_pair: str, quote_pair: str, cross_name: str) -> pd.DataFrame:
    base = df[df["Pair"] == base_pair]
    quote = df[df["Pair"] == quote_pair]

    if base.empty or quote.empty:
        print(f"Missing pair(s): {base_pair}, {quote_pair}")
        return df

    cross_bid = base["Bid"].values[0] / quote["Ask"].values[0]
    cross_ask = base["Ask"].values[0] / quote["Bid"].values[0]

    new_row = pd.DataFrame([{
        "Pair": cross_name,
        "Bid": round(cross_bid, 6),
        "Ask": round(cross_ask, 6)
    }])

    return pd.concat([df, new_row], ignore_index=True)

# **Section 2.A – System Parameters**

**POSITIONAL_BIAS**

This dictionary sets the strategic inventory bias:

Bias	Effect on Quotes
- long	Tighter bid, wider ask – encourages buying.
- short	Wider bid, tighter ask – encourages selling.
- neutral	Symmetric spread around mid-price.

It guides how the spread is skewed depending on inventory or market view.

**Market Maker Style Definitions**

The MAKER_STYLE parameter controls the quoting behavior of the FX maker, affecting the width of the bid-ask spread. The spread adjustment is applied to the base spread (defined in TRADE_PARAMS) to derive the final maker quote. Each style reflects a different level of quote aggressiveness:


| Style              | Description                                                                                                         | Spread Adjustment Multiplier |
| ------------------ | ------------------------------------------------------------------------------------------------------------------- | ---------------------------- |
| `super_aggressive` | Quotes extremely tight spreads to always be at top-of-book, maximizing fills but increasing adverse selection risk. | `0.25 × base spread`         |
| `ultra_aggressive` | Still very competitive, close to market with slightly more risk moderation.                                         | `0.4 × base spread`          |
| `aggressive`       | Default aggressive quoting behavior, aiming to be competitive.                                                      | `0.5 × base spread`          |
| `neutral`          | No adjustment; uses the base spread as-is.                                                                          | `1.0 × base spread`          |
| `defensive`        | Widens quotes to reduce fill probability, suitable for cautious risk stance.                                        | `1.5 × base spread`          |


In [8]:
TRADE_PARAMS = {
    "AUD/USD": {
        "max_spread_bps": 20,
        "pip_size": 0.0001,
        "min_size": 5_000_000,
        "max_size": 20_000_000
    },
    "AUD/EUR": {
        "max_spread_bps": 20,
        "pip_size": 0.0001,
        "min_size": 5_000_000,
        "max_size": 20_000_000
    },
    "AUD/GBP": {
        "max_spread_bps": 20,
        "pip_size": 0.0001,
        "min_size": 5_000_000,
        "max_size": 20_000_000
    },
    "USD/JPY": {
        "max_spread_bps": 40,
        "pip_size": 0.01,
        "min_size": 5_000_000,
        "max_size": 20_000_000
    },
    "EUR/USD": {
        "max_spread_bps": 20,
        "pip_size": 0.0001,
        "min_size": 5_000_000,
        "max_size": 20_000_000
    }
}

#Long, Neutral, Short

POSITIONAL_BIAS = {
    "AUD/USD": "long",
    "AUD/EUR": "neutral",
    "USD/JPY": "neutral",
    "USD/EUR": "neutral"
}

# 'super_aggressive', 'ultra_aggressive', 'aggressive', 'neutral', 'defensive'
MAKER_STYLE = "defensive"



# **Market maker: Hub (quote generation)**

In [9]:
def generate_maker_quote(pair: str, bid: float, ask: float) -> dict:
    params = TRADE_PARAMS.get(pair)
    if not params:
        raise ValueError(f"No trading params defined for {pair}.")

    pip = params["pip_size"]
    max_spread_decimal = params["max_spread_bps"] * 0.0001  # bps → decimal

    # Adjust style
    if MAKER_STYLE == "super_aggressive":
        style_multiplier = 0.25
    elif MAKER_STYLE == "ultra_aggressive":
        style_multiplier = 0.4
    elif MAKER_STYLE == "aggressive":
        style_multiplier = 0.5
    elif MAKER_STYLE == "defensive":
        style_multiplier = 1.5
    else:
        style_multiplier = 1.0

    final_spread = max_spread_decimal * style_multiplier
    half_spread = final_spread / 2
    mid_price = (bid + ask) / 2
    bias = POSITIONAL_BIAS.get(pair, "neutral")

    # Quote construction
    if bias == "long":
        maker_bid = mid_price - half_spread * 0.8
        maker_ask = mid_price + half_spread * 1.2
    elif bias == "short":
        maker_bid = mid_price - half_spread * 1.2
        maker_ask = mid_price + half_spread * 0.8
    else:
        maker_bid = mid_price - half_spread
        maker_ask = mid_price + half_spread

    # Ensure spread not exceeded
    spread_check = maker_ask - maker_bid
    if spread_check > max_spread_decimal:
        # Clamp ask to stay within max spread
        maker_ask = maker_bid + max_spread_decimal

    return {
        "Pair": pair,
        "Market Bid": round(bid, 6),
        "Market Ask": round(ask, 6),
        "Maker Bid": round(maker_bid, 6),
        "Maker Ask": round(maker_ask, 6),
        "Spread (bps)": round((maker_ask - maker_bid) / pip * 0.01, 2),
        "Max Size": params["max_size"],
        "Min Size": params["min_size"]
    }


In [12]:
def run_maker_quotes(pairs: List[str]) -> pd.DataFrame:
    synthetic_pairs = {
        "AUD/EUR": ("AUD/USD", "EUR/USD"),
        "USD/EUR": ("USD/JPY", "EUR/JPY")
    }

    api_pairs = set()
    for p in pairs:
        if p in synthetic_pairs:
            num, denom = synthetic_pairs[p]
            api_pairs.add(num.replace("/", "_"))
            api_pairs.add(denom.replace("/", "_"))
        else:
            api_pairs.add(p.replace("/", "_"))

    df_quotes = get_oanda_bid_ask(list(api_pairs))
    if df_quotes.empty or "Pair" not in df_quotes.columns:
        print("Error: No valid quote data returned from OANDA.")
        return pd.DataFrame()

    for sp, (num, denom) in synthetic_pairs.items():
        if sp in pairs:
            df_quotes = compute_cross_rate(df_quotes, num.replace("_", "/"), denom.replace("_", "/"), sp)

    quote_results = []
    for pair in pairs:
        row = df_quotes[df_quotes["Pair"] == pair]
        if not row.empty:
            result = generate_maker_quote(pair, row["Bid"].values[0], row["Ask"].values[0])
            quote_results.append(result)

    return pd.DataFrame(quote_results)


In [13]:
def print_subsection(pair_code, subsection_label):
    print(f"\n--- Subsection 7.{subsection_label}: {pair_code} ---")
    df = run_maker_quotes([pair_code])
    if not df.empty:
        print("[Maker Quote]")
        print(df[["Pair", "Maker Bid", "Maker Ask"]].to_string(index=False))
        print("\n[Trade Parameters]")
        print(df[["Max Size", "Min Size"]].to_string(index=False))
        print("\n[Market Quote]")
        print(df[["Market Bid", "Market Ask"]].to_string(index=False))


## **Quotation style**

    Options: 'super_aggressive', 'ultra_aggressive', 'aggressive', 'neutral', 'defensive'

In [14]:
MAKER_STYLE = "aggressive"

## **AUD/USD**

In [15]:
print_subsection("AUD/USD", "A")


--- Subsection 7.A: AUD/USD ---
[Maker Quote]
   Pair  Maker Bid  Maker Ask
AUD/USD    0.65405    0.65505

[Trade Parameters]
 Max Size  Min Size
 20000000   5000000

[Market Quote]
 Market Bid  Market Ask
    0.65441     0.65449


## **USD/JPY**

In [16]:
print_subsection("USD/JPY", "B")


--- Subsection 7.B: USD/JPY ---
[Maker Quote]
   Pair  Maker Bid  Maker Ask
USD/JPY    144.907    144.909

[Trade Parameters]
 Max Size  Min Size
 20000000   5000000

[Market Quote]
 Market Bid  Market Ask
    144.902     144.914


## **AUD/EUR**

In [17]:
print_subsection("AUD/EUR", "C")


--- Subsection 7.C: AUD/EUR ---
[Maker Quote]
   Pair  Maker Bid  Maker Ask
AUD/EUR    0.55721    0.55821

[Trade Parameters]
 Max Size  Min Size
 20000000   5000000

[Market Quote]
 Market Bid  Market Ask
   0.557659    0.557762


## **EUR/USD**

In [18]:
print_subsection("EUR/USD", "D")


--- Subsection 7.D: EUR/USD ---
[Maker Quote]
   Pair  Maker Bid  Maker Ask
EUR/USD   1.172945   1.173945

[Trade Parameters]
 Max Size  Min Size
 20000000   5000000

[Market Quote]
 Market Bid  Market Ask
    1.17338     1.17351


# **Price taker: Hub**

In [19]:
def get_fx_quotes(pairs: Union[str, List[str]]) -> pd.DataFrame:
    # Convert and clean input
    pairs = [pairs] if isinstance(pairs, str) else pairs
    synthetic_map = {
        "AUD/EUR": ("AUD/USD", "EUR/USD"),
        "USD/EUR": ("USD/JPY", "EUR/JPY")
    }

    api_pairs = set()
    for p in pairs:
        if p in synthetic_map:
            num, denom = synthetic_map[p]
            api_pairs.update({num.replace("/", "_"), denom.replace("/", "_")})
        else:
            api_pairs.add(p.replace("/", "_"))

    df = get_oanda_bid_ask(list(api_pairs))
    for p in pairs:
        if p in synthetic_map:
            df = compute_cross_rate(df, synthetic_map[p][0], synthetic_map[p][1], p)

    return df[df["Pair"].isin(pairs)].reset_index(drop=True)


def get_all_fx_quotes() -> pd.DataFrame:
    full_list = ["AUD/USD", "EUR/USD", "USD/JPY", "GBP/USD", "EUR/JPY", "AUD/EUR", "USD/EUR"]
    return get_fx_quotes(full_list)

## **All market quotes**

In [20]:
# 🔹 Tutte le principali + cross sintetici
print(get_all_fx_quotes())

      Pair         Bid         Ask
0  GBP/USD    1.359610    1.359780
1  EUR/JPY  170.023000  170.061000
2  AUD/USD    0.654420    0.654500
3  EUR/USD    1.173290    1.173400
4  USD/JPY  144.914000  144.928000
5  AUD/EUR    0.557713    0.557833
6  USD/EUR    0.852130    0.852402


## **AUD/USD**

In [21]:
print(get_fx_quotes("AUD/USD"))

      Pair      Bid      Ask
0  AUD/USD  0.65446  0.65453


## **USD/JPY**

In [22]:
print(get_fx_quotes("USD/JPY"))

      Pair      Bid      Ask
0  USD/JPY  144.946  144.959


## **AUD/EUR**

In [23]:
print(get_fx_quotes("AUD/EUR"))

      Pair       Bid       Ask
0  AUD/EUR  0.557711  0.557831


## **EUR/USD**

In [24]:
print(get_fx_quotes("EUR/USD"))

      Pair      Bid      Ask
0  EUR/USD  1.17325  1.17336


# Arbitrage

In [26]:
import pandas as pd

# --- CONFIGURATION ---
TRADE_PARAMS = {
    "AUD/USD": {
        "base_currency": "AUD",
        "max_spread_bps": 20,
        "pip_size": 0.0001,
        "min_size": 5_000_000,
        "max_size": 20_000_000
    },
    "AUD/EUR": {
        "base_currency": "AUD",
        "max_spread_bps": 20,
        "pip_size": 0.0001,
        "min_size": 5_000_000,
        "max_size": 20_000_000
    },
    "EUR/USD": {
        "base_currency": "EUR", # The constraint size applies to EUR
        "pip_size": 0.0001,
        "min_size": 5_000_000,
        "max_size": 20_000_000
    }
}


# --- FUNCTIONS ---

def manual_quote_input():
    """Asks the user for quotes for the required currency pairs."""
    # MODIFIED: Asking for EUR/USD instead of USD/EUR
    print("🔢 Enter bid/ask for 3 FX pairs (AUD/USD, EUR/USD, AUD/EUR):")
    pairs = ["AUD/USD", "EUR/USD", "AUD/EUR"]
    data = []
    for p in pairs:
        while True:
            try:
                bid = float(input(f"   ➤ {p} Bid: "))
                ask = float(input(f"   ➤ {p} Ask: "))
                if bid > ask:
                    print("ERROR: The Bid price cannot be higher than the Ask price. Please try again.")
                    continue
                data.append({"Pair": p, "Bid": bid, "Ask": ask})
                break
            except ValueError:
                print("ERROR: Please enter a valid number.")
    return pd.DataFrame(data)

def compute_implied_cross(df):
    """
    Calculates the implied AUD/EUR cross rate from AUD/USD and EUR/USD.
    [LOGIC CHANGED]: With AUD/USD and EUR/USD, the formula is division.
    - Implied Bid = Sell AUD for EUR = (Sell AUD for USD) / (Buy EUR with USD) = AUD/USD Bid / EUR/USD Ask
    - Implied Ask = Buy AUD with EUR = (Buy AUD with USD) / (Sell EUR for USD) = AUD/USD Ask / EUR/USD Bid
    """
    audusd = df.loc[df["Pair"] == "AUD/USD"]
    eurusd = df.loc[df["Pair"] == "EUR/USD"] # Changed from usdeur

    imp_bid = audusd["Bid"].values[0] / eurusd["Ask"].values[0]
    imp_ask = audusd["Ask"].values[0] / eurusd["Bid"].values[0]

    return round(imp_bid, 5), round(imp_ask, 5)

def determine_max_notional(arbitrage_type, rates):
    """Calculates max tradable notional based on new trade structure."""
    audusd_params = TRADE_PARAMS["AUD/USD"]
    audeur_params = TRADE_PARAMS["AUD/EUR"]
    eurusd_params = TRADE_PARAMS["EUR/USD"]

    min_from_audusd, max_from_audusd = audusd_params["min_size"], audusd_params["max_size"]
    min_from_audeur, max_from_audeur = audeur_params["min_size"], audeur_params["max_size"]
    min_eur_traded, max_eur_traded = eurusd_params["min_size"], eurusd_params["max_size"]

    if arbitrage_type == 'MINE': # Sell Implied (Sell AUD -> Get USD -> Use USD to Buy EUR)
        # Amount of EUR traded = (notional_aud * audusd_bid) / eurusd_ask
        # So, notional_aud = (eur_traded * eurusd_ask) / audusd_bid
        min_from_eurusd = (min_eur_traded * rates['eurusd_ask']) / rates['audusd_bid']
        max_from_eurusd = (max_eur_traded * rates['eurusd_ask']) / rates['audusd_bid']

    elif arbitrage_type == 'YOURS': # Buy Implied (Sell EUR -> Get USD -> Use USD to Buy AUD)
        # Amount of EUR traded = (notional_aud * audusd_ask) / eurusd_bid
        # So, notional_aud = (eur_traded * eurusd_bid) / audusd_ask
        min_from_eurusd = (min_eur_traded * rates['eurusd_bid']) / rates['audusd_ask']
        max_from_eurusd = (max_eur_traded * rates['eurusd_bid']) / rates['audusd_ask']

    final_min_notional = max(min_from_audusd, min_from_audeur, min_from_eurusd)
    final_max_notional = min(max_from_audusd, max_from_audeur, max_from_eurusd)

    if final_max_notional < final_min_notional:
        return None
    return final_max_notional

def detect_and_execute_arbitrage(df):
    """Detects and executes arbitrage with the updated EUR/USD logic."""
    imp_bid, imp_ask = compute_implied_cross(df)
    direct = df.loc[df["Pair"] == "AUD/EUR"]
    avv_bid = direct["Bid"].values[0]
    avv_ask = direct["Ask"].values[0]

    print(f"\nImplied AUD/EUR: {imp_bid:.5f} / {imp_ask:.5f}")
    print(f"Direct AUD/EUR: {avv_bid:.5f} / {avv_ask:.5f}\n")

    pip = TRADE_PARAMS["AUD/EUR"]["pip_size"]

    rates = {
        'audusd_bid': df.loc[df['Pair'] == 'AUD/USD', 'Bid'].values[0],
        'audusd_ask': df.loc[df['Pair'] == 'AUD/USD', 'Ask'].values[0],
        'eurusd_bid': df.loc[df['Pair'] == 'EUR/USD', 'Bid'].values[0], # New
        'eurusd_ask': df.loc[df['Pair'] == 'EUR/USD', 'Ask'].values[0], # New
        'audeur_bid': avv_bid,
        'audeur_ask': avv_ask,
    }

    if imp_bid > avv_ask:
        # Opportunity: Sell implied at high price, Buy direct at low price
        notional = determine_max_notional('MINE', rates)
        if notional is None:
            print("Theoretical opportunity exists, but it's not executable due to size constraints.")
            return

        profit_pips = round((imp_bid - avv_ask) / pip, 1)
        print("Arbitrage Opportunity Detected → MINE")
        print("Direction: BUY from market (the direct AUD/EUR rate)")

        # Execute new trade legs
        usd_received = notional * rates['audusd_bid']
        eur_received_synthetic = usd_received / rates['eurusd_ask']
        eur_paid_direct = notional * rates['audeur_ask']
        profit = eur_received_synthetic - eur_paid_direct

        print(f"📉 Trade 1: YOURS (Bid) (AUD/USD) → {rates['audusd_bid']:.5f} | SELL {notional:,.0f} AUD → RECEIVE {usd_received:,.2f} USD")
        print(f"📈 Trade 2: MINE (Ask) (EUR/USD) → {rates['eurusd_ask']:.5f} | BUY {eur_received_synthetic:,.2f} EUR → PAY {usd_received:,.2f} USD")
        print(f"📈 Trade 3: MINE (Ask) (AUD/EUR) → {rates['audeur_ask']:.5f} | BUY {notional:,.0f} AUD → PAY {eur_paid_direct:,.2f} EUR")
        print(f"💰 Profit: {profit_pips} pips | ≈ {profit:,.2f} EUR")

    elif imp_ask < avv_bid:
        # Opportunity: Buy implied at low price, Sell direct at high price
        notional = determine_max_notional('YOURS', rates)
        if notional is None:
            print("Theoretical opportunity exists, but it's not executable due to size constraints.")
            return

        profit_pips = round((avv_bid - imp_ask) / pip, 1)
        print("Arbitrage Opportunity Detected → YOURS")
        print("Direction: SELL to market (the direct AUD/EUR rate)")

        # Execute new trade legs
        usd_needed_for_aud = notional * rates['audusd_ask']
        eur_to_sell_for_usd = usd_needed_for_aud / rates['eurusd_bid']
        eur_received_direct = notional * rates['audeur_bid']
        profit = eur_received_direct - eur_to_sell_for_usd

        print(f"Trade 1: YOURS (Bid) (EUR/USD) → {rates['eurusd_bid']:.5f} | SELL {eur_to_sell_for_usd:,.2f} EUR → RECEIVE {usd_needed_for_aud:,.2f} USD")
        print(f"Trade 2: MINE (Ask) (AUD/USD) → {rates['audusd_ask']:.5f} | BUY {notional:,.0f} AUD → PAY {usd_needed_for_aud:,.2f} USD")
        print(f"Trade 3: YOURS (Bid) (AUD/EUR) → {rates['audeur_bid']:.5f} | SELL {notional:,.0f} AUD → RECEIVE {eur_received_direct:,.2f} EUR")
        print(f"Profit: {profit_pips} pips | ≈ {profit:,.2f} EUR")
    else:
        print("No Arbitrage Opportunity Detected.")


# === SCRIPT EXECUTION ===
quotes_df = manual_quote_input()
detect_and_execute_arbitrage(quotes_df)

🔢 Enter bid/ask for 3 FX pairs (AUD/USD, EUR/USD, AUD/EUR):
   ➤ AUD/USD Bid: 0.65405
   ➤ AUD/USD Ask: 0.65505
   ➤ EUR/USD Bid: 1.1730
   ➤ EUR/USD Ask: 1.1740
   ➤ AUD/EUR Bid: 0.56
   ➤ AUD/EUR Ask: 0.562

Implied AUD/EUR: 0.55711 / 0.55844
Direct AUD/EUR: 0.56000 / 0.56200

Arbitrage Opportunity Detected → YOURS
Direction: SELL to market (the direct AUD/EUR rate)
Trade 1: YOURS (Bid) (EUR/USD) → 1.17300 | SELL 11,168,797.95 EUR → RECEIVE 13,101,000.00 USD
Trade 2: MINE (Ask) (AUD/USD) → 0.65505 | BUY 20,000,000 AUD → PAY 13,101,000.00 USD
Trade 3: YOURS (Bid) (AUD/EUR) → 0.56000 | SELL 20,000,000 AUD → RECEIVE 11,200,000.00 EUR
Profit: 15.6 pips | ≈ 31,202.05 EUR
